<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Modelos seq2seq 2**

Cuando la entrada de cada paso del decodificador proviene de la salida del paso anterior, estamos hablando de un modelo Seq2Seq con realimentación (feedback). Esto es especialmente común en tareas como la generación de texto.

1. **Inicio de la Secuencia**: Se inicia la generación con un token especial, como `<SOS>` (Start of Sequence).

2. **Generación Paso a Paso**:
   - En el primer paso, el decodificador recibe el `<SOS>` y el estado del codificador como entrada.
   - El decodificador procesa esta entrada y genera una salida para este paso.
   - La salida generada se convierte en la entrada para el siguiente paso, junto con el estado actualizado del decodificador.
   - Este proceso se repite hasta que se genera un token especial de fin de secuencia (`<EOS>`, End of Sequence) o hasta alcanzar un límite máximo de longitud.

3. **Ventajas y Desventajas**:
   - **Ventajas**: Esta forma de generar secuencias puede ayudar a mantener la coherencia en las secuencias generadas, ya que cada nueva palabra o token tiene en cuenta lo que ya se ha generado.
   - **Desventajas**: Puede ser más lento, ya que cada paso depende del anterior, y errores en un paso pueden propagarse y afectar los pasos siguientes.


Imagina que tienes un modelo seq2seq entrenado para traducir inglés a español. Para la frase "How are you?", el proceso sería algo así:

1. El codificador procesa "How are you?" y genera un vector latente.
2. El decodificador recibe el vector latente y el token `<SOS>`.
3. El decodificador genera "¿Cómo", actualiza su estado y toma "¿Cómo" como entrada para el siguiente paso.
4. El decodificador genera "estás", actualiza su estado y toma "estás" como entrada para el siguiente paso.
5. Y así sucesivamente, hasta generar `<EOS>` para indicar el final de la secuencia.

Este modelo de generación permite que el decodificador tenga en cuenta no solo el contexto proporcionado por el codificador, sino también la estructura de la secuencia que está generando, paso a paso.


<p align="center">
<img src="./imgs/seq2seq_feedback.svg" width="70%">
</p>


### **Teacher Forcing**

**Teacher forcing** es una técnica utilizada en el entrenamiento de modelos seq2seq en la que, en un porcentaje de las veces, se utiliza la salida real (etiqueta) de un paso de tiempo como entrada para el siguiente paso, en lugar de la salida predicha por el modelo. Esta técnica puede ayudar a acelerar la convergencia y mejorar el rendimiento del modelo, especialmente en las etapas iniciales del entrenamiento.

#### **¿Cómo funciona?**

1. **Durante el Entrenamiento**: En cada paso de tiempo y con una cierta probabilidad de que suceda, en lugar de pasar la predicción del modelo del paso anterior como entrada al siguiente paso, se pasa la palabra real de la secuencia objetivo. Esto proporciona al modelo información directa y clara sobre cómo debería haber respondido en el paso anterior, independientemente de si la predicción fue correcta o no.

2. **Durante la Evaluación/Predicción**: El modelo debe generar secuencias por sí mismo, utilizando sus propias predicciones del paso anterior para el siguiente paso. Durante esta fase, no se utiliza "teacher forcing".

#### **Ventajas de Teacher Forcing:**

1. **Aprendizaje más Rápido**: Al proporcionar al modelo la respuesta correcta en cada paso, se reduce la propagación de errores a través de la secuencia, lo que puede llevar a un aprendizaje más rápido.

2. **Mejor Rendimiento**: Puede resultar en un mejor rendimiento del modelo, especialmente en las primeras etapas del entrenamiento.

### **Implementación traductor inglés a español**

#### **Dataset Europarl**

El conjunto de datos Europarl contiene las transcripciones de los procedimientos del Parlamento Europeo, proporcionando una valiosa fuente de textos paralelos en 21 idiomas europeos. Las oraciones están alineadas entre los idiomas, lo que lo hace especialmente útil para tareas de traducción automática. 

Descargamos el dataset Europarl para español-inglés. Una vez descargado tendremos dos ficheros de texto, uno para cada idioma con las frases alineadas. El código siguiente muestra las primeras frases de cada fichero.

In [1]:
def read_translation(archivo_ingles, archivo_espanol):
    with open(archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(archivo_espanol, 'r', encoding='utf-8') as f_espanol:
        for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
            yield oracion_ingles.strip(), oracion_espanol.strip()


archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

# Leer el conjunto de datos
for i, (ingles, espanol) in enumerate(read_translation(archivo_ingles, archivo_espanol)):
    print('Inglés:', ingles)
    print('Español:', espanol)
    print('---')
    if i == 15:
        break

Inglés: Resumption of the session
Español: Reanudación del período de sesiones
---
Inglés: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Español: Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.
---
Inglés: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
Español: Como todos han podido comprobar, el gran "efecto del año 2000" no se ha producido. En cambio, los ciudadanos de varios de nuestros países han sido víctimas de catástrofes naturales verdaderamente terribles.
---
Inglés: You have requested a debate on this subject in the course of the next

<div style="background-color:#e3fcff; padding: 15px; box-shadow: 0px 0px 10px rgba(0,0,0,0.2);">

##### **Nota**

En TorchText, las clases `Vocab` y `Vectors` desempeñan roles distintos en el procesamiento de texto, a pesar de que ambas están relacionadas con la representación y manipulación de palabras. Mientras que `Vocab` se enfoca en la mapeo básico entre tokens e índices numéricos, `Vectors` se ocupa de proporcionar representaciones vectoriales de palabras que capturan aspectos semánticos y relacionales. `Vocab` es fundamental para cualquier tarea de procesamiento de texto que requiera convertir texto a una forma numérica, mientras que `Vectors` es crucial para tareas que se benefician de entender la semántica de las palabras, como en la clasificación de texto o en modelos de lenguaje.

Fíjate que en la siguiente celda de código cargamos:

`self.vocab_es = torchtext.vocab.FastText(language='es', unk_init=torch.Tensor.normal_)`

En este caso, `self.vocab_es` es un objeto de la clase `torchtext.vocab.FastText` que, a su vez, hereda de la clase `torchtext.vocab.Vectors`.

Puedes echar un vistazo a todo esto en: https://pytorch.org/text/stable/vocab.html

</div>

In [2]:
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torchtext
import torch
from collections import defaultdict


class Translation(Dataset):
    def __init__(self, source_file, target_file):
        self.ingles = []
        self.espanol = []
        self.tokenizer_es = get_tokenizer("spacy", language="es_core_news_md")
        self.tokenizer_en = get_tokenizer("spacy", language="en_core_web_md")
        self.vocab_es = torchtext.vocab.FastText(language='es', unk_init=torch.Tensor.normal_)
        self.vocab_en = torchtext.vocab.FastText(language='en', unk_init=torch.Tensor.normal_)

        # Añadir tokens especiales
        self.vocab_en = self.add_sos_eos_unk_pad(self.vocab_en)
        self.vocab_es = self.add_sos_eos_unk_pad(self.vocab_es)

        self.archivo_ingles = source_file
        self.archivo_espanol = target_file

        # Leer el conjunto de datos
        for ingles, espanol in self.read_translation():
            self.ingles.append(ingles)
            self.espanol.append(espanol)


    def add_sos_eos_unk_pad(self, vocabulary):
        words = vocabulary.itos  # itos: índice a string. Es una lista con las palabras del vocabulario
        vocab = vocabulary.stoi  # stoi: string to índice. Es un diccionario palabra -> índice
        embedding_matrix = vocabulary.vectors

        # Tokens especiales
        sos_token = '<sos>'
        eos_token = '<eos>'
        pad_token = '<pad>'
        unk_token = '<unk>'

        # Inicializamos los vectores para los tokens especiales, por ejemplo, con ceros
        sos_vector = torch.full((1, embedding_matrix.shape[1]), 1.)
        eos_vector = torch.full((1, embedding_matrix.shape[1]), 2.)
        pad_vector = torch.zeros((1, embedding_matrix.shape[1]))
        unk_vector = torch.full((1, embedding_matrix.shape[1]), 3.)

        # Añade los vectores al final de la matriz de embeddings
        embedding_matrix = torch.cat((embedding_matrix, sos_vector, eos_vector, unk_vector, pad_vector), 0)

        # Añade los tokens especiales al vocabulario
        vocab[sos_token] = len(vocab)  # len(vocab) es el índice del último token
        vocab[eos_token] = len(vocab)  # Aquí len(vocab) se ha incrementado en 1 después de la línea anterior
        vocab[pad_token] = len(vocab)
        vocab[unk_token] = len(vocab)

        words.append(sos_token)
        words.append(eos_token)
        words.append(pad_token)
        words.append(unk_token)

        vocabulary.itos = words
        vocabulary.stoi = vocab
        vocabulary.vectors = embedding_matrix

        default_stoi = defaultdict(lambda : len(vocabulary)-1, vocabulary.stoi)  # Si hay una palabra que no está en el vocabulario, devolverá el índice de <unk>
        vocabulary.stoi = default_stoi
    
        return vocabulary
        

    # Generador para leer las oraciones en inglés y español
    def read_translation(self):
        with open(self.archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(self.archivo_espanol, 'r', encoding='utf-8') as f_espanol:
            for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
                yield oracion_ingles.strip().lower(), oracion_espanol.strip().lower()

    def __len__(self):
        return len(self.ingles)  # Devuelve el número de oraciones. Ingles y español tienen la misma longitud.

    def __getitem__(self, idx):
        item = self.ingles[idx], self.espanol[idx]
        tokens_ingles = self.tokenizer_en(item[0])
        tokens_espanol = self.tokenizer_es(item[1])

        tokens_ingles = tokens_ingles + ['<eos>']
        tokens_espanol = ['<sos>'] + tokens_espanol + ['<eos>']

        if not tokens_ingles or not tokens_espanol:
            return torch.zeros(1, 300), torch.zeros(1, 300)
            # raise RuntimeError("Una de las muestras está vacía.")
    
        tensor_ingles = self.vocab_en.get_vecs_by_tokens(tokens_ingles)
        tensor_espanol = self.vocab_es.get_vecs_by_tokens(tokens_espanol)

        indices_ingles = [self.vocab_en.stoi[token] for token in tokens_ingles] + [self.vocab_en.stoi['<pad>']]
        indices_espanol = [self.vocab_es.stoi[token] for token in tokens_espanol] + [self.vocab_es.stoi['<pad>']]

        return tensor_ingles, tensor_espanol, indices_ingles, indices_espanol
        
            
        
def collate_fn(batch):
    ingles_batch, espanol_batch, ingles_seqs, espanol_seqs = zip(*batch)  # Desempaqueta el batch
    ingles_batch = pad_sequence(ingles_batch, batch_first=True, padding_value=0)
    espanol_batch = pad_sequence(espanol_batch, batch_first=True, padding_value=0)

    # Calcular la longitud máxima de la lista de listas de índices
    pad = espanol_seqs[0][-1]  # token <pad>
    max_len = max([len(l) for l in espanol_seqs])
    for seq in espanol_seqs:
        seq += [pad]*(max_len-len(seq))
        
    return ingles_batch, espanol_batch, ingles_seqs, espanol_seqs

In [3]:
# archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
# archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

archivo_ingles = 'data/mock/mock.en'
archivo_espanol = 'data/mock/mock.es'

translation = Translation(archivo_ingles, archivo_espanol)

### **Modelo**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

##### **Encoder**

In [5]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__() 
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        output, (hidden, cell) = self.rnn(x)
        return output, (hidden, cell)

##### **Decoder**

In [6]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)  # TO DO: Añadir dropout 
        self.fc_out = nn.Linear(hidden_dim, output_dim) # hid_dim(512) -> vocab_size(400_000)

    def forward(self, x, hidden, cell):
        output, (hidden, cell) = self.rnn(x, (hidden, cell))
        output = self.fc_out(output)
        return output, (hidden, cell)

##### **Seq2seq**

In [7]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder                           
        self.es_embeddings = torchtext.vocab.FastText(language='es')
        self.M = self.es_embeddings.vectors
        self.M = torch.cat((self.M, torch.zeros((4, self.M.shape[1]))), 0)

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        target_len = target.shape[1]
        batch_size = target.shape[0]

        # Tensor para almacenar las salidas del decoder
        # outputs = torch.zeros(batch_size, target_len, 985671)
        outputs = torch.zeros(batch_size, target_len, self.es_embeddings.vectors.shape[0] + 4)
        
        # Primero, la fuente es procesada por el encoder
        _, (hidden, cell) = self.encoder(source)

        # La primera entrada al decoder es el vector <sos>
        x = target[:, 0, :]

        for t in range(1, target_len):
            output, (hidden, cell) = self.decoder(x.unsqueeze(1), hidden, cell)
            outputs[:, t, :] = output.squeeze(1)
            
            # Decidir si se va a utilizar teacher forcing. ¿Estamos en modo de entrenamiento?
            if not self.training:
                teacher_forcing_ratio = 0.
            
            teacher_forcing = random.random() < teacher_forcing_ratio
            if teacher_forcing:
                x = target[:, t, :]  # Utilizar la palabra real como siguiente entrada
            else:
                x = torch.matmul(output.squeeze(1), self.M)  # Utilizar la palabra predicha como siguiente entrada

        return outputs

In [8]:
# Parámetros
input_dim = 300
output_dim = translation.vocab_es.vectors.shape[0]
hidden_dim = 512
num_layers = 2
learning_rate = 0.001
num_epochs = 30
batch_size = 8
num_workers = 0
shuffle = True

# Inicializa el modelo, el optimizador y la función de pérdida
encoder = Encoder(input_dim, hidden_dim, num_layers)
decoder = Decoder(input_dim, hidden_dim, output_dim, num_layers)
model = Seq2Seq(encoder, decoder)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# DataLoader
from torch.utils.data import DataLoader

dataloader = DataLoader(translation, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_fn)

In [25]:
# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (src, tgt, src_indices, tgt_indices) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(src, tgt)

        tgt_indices = torch.tensor(tgt_indices, dtype=torch.long)
        loss = 0
        for t in range(1, tgt.shape[1]):
            loss += criterion(output[:, t, :], tgt_indices[:, t])

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_idx % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}')


Epoch [1/30], Step [1/2], Loss: 2.2453
Epoch [1/30], Average Loss: 2.2162
Epoch [2/30], Step [1/2], Loss: 2.1444
Epoch [2/30], Average Loss: 2.1793
Epoch [3/30], Step [1/2], Loss: 2.0332
Epoch [3/30], Average Loss: 2.2329
Epoch [4/30], Step [1/2], Loss: 2.0594
Epoch [4/30], Average Loss: 2.1081
Epoch [5/30], Step [1/2], Loss: 1.9984
Epoch [5/30], Average Loss: 2.0455
Epoch [6/30], Step [1/2], Loss: 1.9262
Epoch [6/30], Average Loss: 1.9919
Epoch [7/30], Step [1/2], Loss: 1.9419
Epoch [7/30], Average Loss: 1.9132
Epoch [8/30], Step [1/2], Loss: 1.8595
Epoch [8/30], Average Loss: 1.9260
Epoch [9/30], Step [1/2], Loss: 1.8338
Epoch [9/30], Average Loss: 1.9036
Epoch [10/30], Step [1/2], Loss: 1.9159
Epoch [10/30], Average Loss: 1.7998
Epoch [11/30], Step [1/2], Loss: 1.8753
Epoch [11/30], Average Loss: 1.7168
Epoch [12/30], Step [1/2], Loss: 1.7385
Epoch [12/30], Average Loss: 1.7632
Epoch [13/30], Step [1/2], Loss: 1.7107
Epoch [13/30], Average Loss: 1.7257
Epoch [14/30], Step [1/2], Los

In [83]:
# Test the model with input sentences
model.eval()

sentence = "my vehicle"

# Convertir a vectores
tokens = translation.tokenizer_en(sentence)
tokens = tokens + ['<eos>']
text_tensor = translation.vocab_en.get_vecs_by_tokens(tokens)
text_tensor = text_tensor.unsqueeze(0)

with torch.no_grad():
    encoder_outputs, (hidden, cell) = model.encoder(text_tensor)

outputs = []

input_token = torch.tensor(translation.vocab_es.stoi['<sos>']).unsqueeze(0)
input_token = translation.vocab_es.vectors[input_token].unsqueeze(0)
    
for _ in range(5):
    with torch.no_grad():
        output, (hidden, cell) = model.decoder(input_token, hidden, cell) # teacher_forcing_ratio=0.0, estamos evaluando, no entrenando.
        
    # Obtener el token con la probabilidad más alta
    best_guess = output.argmax(2).squeeze(0)
    outputs.append(best_guess.item())
        
    # Si el token es <eos>, terminar la traducción
    if best_guess == translation.vocab_es.stoi['<eos>']:
        break
        
    # Utilizar la palabra predicha como la siguiente entrada al decoder
    input_token = translation.vocab_es.vectors[best_guess].unsqueeze(0)
        
# Convertir los índices de salida a palabras
translated_sentence = [translation.vocab_es.itos[idx] for idx in outputs]
    
result = ' '.join(translated_sentence)

print(result)

mi coche <eos>


In [11]:
# Guardar el modelo
torch.save(model.state_dict(), 'seq2seq.pth')

In [12]:
# Load model from file
model.load_state_dict(torch.load('seq2seq.pth'))

<All keys matched successfully>

In [13]:
import torchtext
my_vocab_es = torchtext.vocab.FastText(language='es')